# Quick intro to Kernel Memory: install, upload a doc, ask a question

This notebook shows the basic usage of Kernel Memory, via the serverless client.

First of all, install the Kernel Memory dependency.

In [1]:
#r "nuget: Microsoft.KernelMemory.Core, 0.20.231211.3"

using Microsoft.KernelMemory;

Installed Packages Microsoft.KernelMemory.Core, 0.20.231211.3

For our demo, we use also the "dotenv" nuget, to load our secret credentials from a `.env` file.
Make sure you create your `.env` file, with this content:

> ```
> OPENAI_API_KEY=<your OpenAI API key>
> ```

In [2]:
#r "nuget: dotenv.net, 3.1.3"

dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Installed Packages dotenv.net, 3.1.3

Now let's create an instance of Kernel Memory client using the `KernelMemoryBuilder` to configure our solution. 

In this demo we use OpenAI to calculate embeddings and generate text, and the default storage settings, with content and embeddings kept in a volatile memory automatically deleted after the execution.

In [3]:
var memory = new KernelMemoryBuilder()
    .WithOpenAIDefaults(env["OPENAI_API_KEY"])
    .Build<MemoryServerless>();

info: Microsoft.KernelMemory.Handlers.TextExtractionHandler[0]
      Handler 'extract' ready
info: Microsoft.KernelMemory.Handlers.TextPartitioningHandler[0]
      Handler 'partition' ready
info: Microsoft.KernelMemory.Handlers.SummarizationHandler[0]
      Handler 'summarize' ready
info: Microsoft.KernelMemory.Handlers.GenerateEmbeddingsHandler[0]
      Handler 'gen_embeddings' ready, 1 embedding generators
info: Microsoft.KernelMemory.Handlers.SaveRecordsHandler[0]
      Handler save_records ready, 1 vector storages
info: Microsoft.KernelMemory.Handlers.DeleteDocumentHandler[0]
      Handler 'private_delete_document' ready
info: Microsoft.KernelMemory.Handlers.DeleteIndexHandler[0]
      Handler 'private_delete_index' ready


At this point we have the memory client ready, we can import data and ask questions!

Let's import a PDF file with some details about Semantic Kernel:

In [4]:
await memory.ImportDocumentAsync("sample-SK-Readme.pdf", documentId: "doc001");

Now, let's ask a question about Semantic Kernel:

In [5]:
var question = "What's Semantic Kernel?";

var answer = await memory.AskAsync(question);

Console.WriteLine($"Question: {question}\n\nAnswer: {answer.Result}");

Question: What's Semantic Kernel?

Answer: Semantic Kernel is a lightweight SDK (Software Development Kit) developed by Microsoft. It enables the integration of AI Large Language Models (LLMs) with conventional programming languages. The SDK combines natural language semantic functions, traditional code native functions, and embeddings-based memory to add value and unlock new potential in applications with AI. Semantic Kernel supports prompt templating, function chaining, vectorized memory, and intelligent planning capabilities. It encapsulates several design patterns from the latest AI research, allowing developers to infuse their applications with features like prompt chaining, recursive reasoning, summarization, zero/few-shot learning, contextual memory, long-term memory, embeddings, semantic indexing, planning, retrieval-augmented generation, and accessing external knowledge stores. Semantic Kernel is open-source and aims to enable developers to build AI-first apps faster.


And here show the sources used to generate the answer above, what we call "citations":

In [6]:
Console.WriteLine("Sources:\n");

foreach (var x in answer.RelevantSources)
{
    Console.WriteLine($"  - {x.SourceName}  - {x.Link} [{x.Partitions.First().LastUpdate:D}]");
}

Sources:

  - sample-SK-Readme.pdf  - doc001/ecb6d46eb039411193996b6cc802b11e [Friday, December 8, 2023]
